In [1]:
import os
import sys
import random
import time
from random import seed, randint
import argparse
import platform
from datetime import datetime
import imp
import numpy as np
import fileinput
from itertools import product
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
import seaborn as sns
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib as mpl
sys.path.insert(0,'..')
# from notebookFunctions import *
# from .. import notebookFunctions

%matplotlib inline
plt.rcParams['figure.figsize'] = (10,6.180)    #golden ratio
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/Users/weilu/openmmawsem/')

from openmmawsem import *

In [2]:
os.chdir("/Users/weilu/openmmawsem/_local/from_lammps_to_openmm/")

In [6]:
simulation_platform = "OpenCL" # OpenCL, CUDA, CPU, or Reference
pdb_id = '1r69'
pdb = f"{pdb_id}.pdb"
chain='T'

# download(pdb_id)
os.system("cp /Users/weilu/opt/parameters/globular_parameters/burial_gamma.dat .")
os.system("cp /Users/weilu/opt/parameters/globular_parameters/gamma.dat .")
input_pdb_filename, cleaned_pdb_filename = prepare_pdb(pdb, chain)
# ensure_atom_order(input_pdb_filename)
# getSeqFromCleanPdb(input_pdb_filename, chains='A')

In [ ]:
ensure_atom_order(input_pdb_filename)

In [158]:
pwd

'/Users/weilu/openmmawsem/_local/from_lammps_to_openmm'

In [13]:
a = open(pdb).read().split("END")

In [143]:
os.system("rm openmmMovie.pdb")
os.system(f"echo 'REMARK converted from awsem lammps output' >> openmmMovie.pdb")
for i in range(30):
    with open("tmp.pdb", "w") as out:
        out.write(a[i])
    input_pdb_filename, cleaned_pdb_filename =prepare_pdb("tmp.pdb", chain)
    os.system(f"echo 'MODEL  {i+1}' >> openmmMovie.pdb")
    os.system("cat tmp-openmmawsem.pdb >> openmmMovie.pdb")

In [159]:
pdb_trajectory = read_trajectory_pdb_positions("openmmMovie.pdb")
oa = OpenMMAWSEMSystem(input_pdb_filename, k_awsem=1.0, xml_filename="../../awsem.xml") # k_awsem is an overall scaling factor that will affect the relevant temperature scales

# apply forces
# forceGroupTable_Rev = {11:"Con", 12:"Chain", 13:"Chi", 14:"Excluded", 15:"Rama", 16:"Direct", 
#                   17:"Burial", 18:"Mediated", 19:"Fragment"}
forceGroupTable = {"Con":11, "Chain":12, "Chi":13, "Excluded":14, "Rama":15, "Direct":16,
                    "Burial":17, "Mediated":18, "Contact":18, "Fragment":19, "Total":list(range(11, 20)),
                    "Water":[16, 18], "Q":1}
forces = [
    oa.q_value("crystal_structure-cleaned.pdb", "A"),
#     oa.con_term(),
    oa.chain_term(),
    oa.chi_term(),
#     oa.excl_term(),
    oa.rama_term(),
    oa.rama_proline_term(),
    oa.direct_term(),
    oa.burial_term(),
    oa.mediated_term(),
#     oa.fragment_memory_term(frag_location_pre="./")
]
oa.addForcesWithDefaultForceGroup(forces)

# start simulation
collision_rate = 5.0 / picoseconds

integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 2*femtoseconds)
simulation = Simulation(oa.pdb.topology, oa.system, integrator, Platform.getPlatformByName("OpenCL"))

In [160]:
showEnergy = ["Q", "Con", "Chain", "Chi", "Excluded", "Rama", "Water", "Burial", "Fragment", "Total"]
# print("Steps", *showEnergy)
print(" ".join(["{0:<8s}".format(i) for i in ["Steps"] + showEnergy]))
for step, pdb in enumerate(pdb_trajectory):
    simulation.context.setPositions(pdb.positions)
    e = []
    for term in showEnergy:
        if type(forceGroupTable[term]) == list:
            g = set(forceGroupTable[term])
        elif forceGroupTable[term] == -1:
            g = -1
        else:
            g = {forceGroupTable[term]}
        state = simulation.context.getState(getEnergy=True, groups=g)
        termEnergy = state.getPotentialEnergy().value_in_unit(kilocalories_per_mole)
        e.append(termEnergy)
#     print(*e)
    print(" ".join([f"{step:<8}"] +  ["{0:<8.2f}".format(i) for i in e]))
#         print(forceGroupTable[term], state.getPotentialEnergy().value_in_unit(kilocalories_per_mole))

Steps    Q        Con      Chain    Chi      Excluded Rama     Water    Burial   Fragment Total   
0        0.00     0.00     20.00    1.34     0.00     -133.72  0.00     -34.38   0.00     -146.76 
1        0.04     0.00     564.83   104.41   0.00     -64.29   -0.15    -50.97   0.00     553.83  
2        0.07     0.00     498.49   64.70    0.00     -63.21   -6.54    -53.70   0.00     439.74  
3        0.07     0.00     322.47   41.74    0.00     -79.27   -0.94    -51.73   0.00     232.27  
4        0.08     0.00     333.68   65.62    0.00     -99.04   -8.78    -55.24   0.00     236.24  
5        0.09     0.00     296.15   62.99    0.00     -105.74  -5.32    -53.39   0.00     194.70  
6        0.10     0.00     351.38   63.06    0.00     -93.40   -3.81    -54.43   0.00     262.79  
7        0.09     0.00     291.82   41.01    0.00     -107.20  -2.96    -54.26   0.00     168.40  
8        0.11     0.00     269.54   45.22    0.00     -101.12  -2.05    -53.24   0.00     158.35  
9        0

In [147]:
124+284+51+9-119-21-54-305

-31

In [149]:
284+51-117-21-54-305+127+9

-26

In [89]:
for pdb in pdb_trajectory:
    simulation.context.setPositions(pdb.positions)
    
    state = simulation.context.getState(getEnergy=True, groups={2})
    print(state.getPotentialEnergy().value_in_unit(kilocalories_per_mole))

0.0
0.0
0.0


In [65]:
for pdb in pdb_trajectory:
    simulation.context.setPositions(pdb.positions)
    
    state = simulation.context.getState(getEnergy=True, groups={2})
    print(state.getPotentialEnergy().value_in_unit(kilocalories_per_mole))

19.998466535462477
564.831239543499
498.48847544216056


In [ ]:
order_parameters_to_compute = {
#     "Qvalue": oa.q_value(qvalue_reference_structure, qvalue_reference_chain, min_seq_sep=3, max_seq_sep=np.inf), 
    "Con": oa.con_term(),
    "Chain": oa.chain_term(),
    "Chi": oa.chi_term(),
}


In [15]:
a[1]

'\nATOM      1  CA  SER T   1     -25.844 -25.886   4.088  1.00  0.00           C\nATOM      2  C   SER T   1     -25.118 -27.205   4.034  1.00  0.00           C\nATOM      3  O   SER T   1     -25.635 -28.157   4.488  1.00  0.00           O\nATOM      4  CB  SER T   1     -25.753 -25.059   5.387  1.00  0.00           C\nATOM      5  N   ILE T   2     -23.914 -27.226   3.469  1.00  0.00           N\nATOM      6  CA  ILE T   2     -23.043 -28.392   3.314  1.00  0.00           C\nATOM      7  C   ILE T   2     -22.019 -28.229   4.438  1.00  0.00           C\nATOM      8  O   ILE T   2     -21.345 -27.296    4.45  1.00  0.00           O\nATOM      9  CB  ILE T   2     -22.543 -28.281   1.795  1.00  0.00           C\nATOM     10  N   SER T   3     -21.929 -29.161   5.373  1.00  0.00           N\nATOM     11  CA  SER T   3     -21.007 -29.194   6.543  1.00  0.00           C\nATOM     12  C   SER T   3     -19.659 -28.682   6.455  1.00  0.00           C\nATOM     13  O   SER T   3     -18.92

In [1]:
input_pdb_filename = "/Users/weilu/Research/server/dec_2018/T0958_single_chain/openmm_T0958/T0958-openmmawsem.pdb"

In [60]:
data = pd.read_table(input_pdb_filename, sep="\s+", header=None, names=["_","i","type","res","chain","res_id", "x","y","z","_1","_2","_3"])
data = data.dropna().reset_index()
data["res_id"] = data["res_id"].apply(lambda x: int(x))


In [56]:
def compute_chi(data):
    ca_all = data.query("type == 'CA'")[["x","y","z"]].values
    cb_all = data.query("type == 'CB'")[["x","y","z"]].values
    c_all = data.query("type == 'C'")[["x","y","z"]].values
    n_all = data.query("type == 'N'")[["x","y","z"]].values
    energy = 0 
    for i in range(len(n_all)):
        ca = ca_all[i]
        cb = cb_all[i]
        c = c_all[i]
        n = n_all[i]
        chi0 = -0.71
        k_chi = 60*4.184
        r_ca_cb = cb-ca
        r_c_ca = ca-c
        r_ca_n = n-ca
        norm_r_ca_cb = np.sum(r_ca_cb**2)**0.5
        norm_r_c_ca = np.sum(r_c_ca**2)**0.5
        norm_r_ca_n = np.sum(r_ca_n**2)**0.5
        a = np.cross(-r_c_ca,r_ca_n)/norm_r_c_ca/norm_r_ca_n
        chi = np.dot(a,r_ca_cb)/norm_r_ca_cb
        dchi = chi - chi0
        energy += k_chi*dchi*dchi
    return energy

In [62]:
chosen = data.query("res != 'IGL' and res_id != 1 and res_id != 63")
compute_chi(chosen)/4.184

64.70332860251312

In [61]:
data

,index,_,i,type,res,chain,res_id,x,y,z,_1,_2,_3
0,0,ATOM,1.0,CA,NGP,A,1,-4.514,-24.801,10.243,1.0,0.0,C
1,1,ATOM,2.0,C,NGP,A,1,-5.909,-25.417,10.716,1.0,0.0,C
2,2,ATOM,3.0,O,NGP,A,1,-6.285,-25.465,11.924,1.0,0.0,O
3,3,ATOM,4.0,CB,NGP,A,1,-3.986,-23.638,10.995,1.0,0.0,B
4,4,ATOM,5.0,N,NGP,A,2,-6.656,-25.883,9.731,1.0,0.0,N
5,5,ATOM,6.0,H,NGP,A,2,-6.353,-25.845,8.757,1.0,0.0,H
6,6,ATOM,7.0,CA,NGP,A,2,-8.029,-26.514,9.960,1.0,0.0,C
7,7,ATOM,8.0,C,NGP,A,2,-8.095,-27.697,8.943,1.0,0.0,C
8,8,ATOM,9.0,O,NGP,A,2,-7.165,-28.142,8.443,1.0,0.0,O
9,9,ATOM,10.0,CB,NGP,A,2,-9.303,-25.503,9.633,1.0,0.0,B


In [162]:
pdb_file = "/Users/weilu/openmmawsem/_local/run_openmm_awsem/crystal_structure-cleaned.pdb"
chain_name = "A"
min_seq_sep=3
max_seq_sep=np.inf
contact_threshold=0.8*nanometers
structure_interactions = []
parser = PDBParser()
structure = parser.get_structure('X', pdb_file)
chain = structure[0][chain_name]
residues = [x for x in chain]
for i, residue_i in enumerate(residues):
    for j, residue_j in enumerate(residues):
        ca_list = []
        cb_list = []
        atom_list_i = []
        atom_list_j = []
        if i-j >= min_seq_sep and i-j <= max_seq_sep:  # taking the signed value to avoid double counting
            ca_i = residue_i['CA']
            ca_list.append(ca_i)
            atom_list_i.append(ca_i)
            ca_j = residue_j['CA']
            ca_list.append(ca_j)
            atom_list_j.append(ca_j)
            if not residue_i.get_resname() == "GLY":
                cb_i = residue_i['CB']
                cb_list.append(cb_i)
                atom_list_i.append(cb_i)
            if not residue_j.get_resname() == "GLY":
                cb_j = residue_j['CB']
                cb_list.append(cb_j)
                atom_list_j.append(cb_j)
            for atom_i, atom_j in product(atom_list_i, atom_list_j):
                r_ijN = abs(atom_i - atom_j)/10.0*nanometers # convert to nm
                
                if r_ijN <= contact_threshold:
                    sigma_ij = 0.1*abs(i-j)**0.15 # 0.1 nm = 1 A
                    gamma_ij = 1.0
#                     if atom_i in ca_list:
#                         i_index = self.ca[i]
#                     if atom_i in cb_list:
#                         i_index = self.cb[i]
#                     if atom_j in ca_list:
#                         j_index = self.ca[j]
#                     if atom_j in cb_list:
#                         j_index = self.cb[j]
#                     structure_interaction = [i_index, j_index, [gamma_ij, r_ijN, sigma_ij]]
#                     structure_interactions.append(structure_interaction)



In [168]:
for i, residue_i in enumerate(residues):
    print(residue_i['CA'])

<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>
<Atom CA>


In [5]:
a = getSeqFromCleanPdb("/Users/weilu/openmmawsem/_local/2fha_openmm/2fha/2fha-openmmawsem.pdb")

In [6]:
len(a)

150

In [8]:

# determine number of rows to skip
skipRows = 0
found = False
with open(cleaned_pdb_filename) as f:
    for line in f:
        if len(line) > 5:
            if line[:4] == "ATOM":
                found = True
                break
        skipRows += 1
if not found:
    print("No ATOM found")
a = pd.read_table(cleaned_pdb_filename, skiprows=skipRows, sep="\s+", names=["ATOM", "i", "Type", "Res", "Chain", "ResId", "x", "y", "z", "_", "_1", "_2"]).dropna()

# save chain seq to pdb.fasta
import textwrap
with open(fastaFile, "w") as out:
    for chain in chains:
        out.write(f">{pdb.upper()}:{chain.upper()}|PDBID|CHAIN|SEQUENCE\n")
        threeLetterSeq = a.query(f"Chain == '{chain}' and Type == 'CA'")["Res"]
        chain_seq = "".join([ThreeToOne[i] for i in threeLetterSeq])
        out.write("\n".join(textwrap.wrap(chain_seq, width=80))+"\n")
threeLetterSeq = a.query("Type == 'CA'")["Res"]
seq = "".join([ThreeToOne[i] for i in threeLetterSeq])

In [11]:
parser = PDBParser()

In [30]:
input_pdb_filename = "/Users/weilu/openmmawsem/_local/2fha_openmm/2fha/2fha-openmmawsem.pdb"
chains = "A"
cleaned_pdb_filename = input_pdb_filename.replace("openmmawsem.pdb", "cleaned.pdb")
pdb = input_pdb_filename.replace("-openmmawsem.pdb", "")
fastaFile = pdb + ".fasta"
ThreeToOne = {'ALA':'A','ARG':'R','ASN':'N','ASP':'D','CYS':'C','GLU':'E','GLN':'Q','GLY':'G','HIS':'H',
       'ILE':'I','LEU':'L','LYS':'K','MET':'M','PHE':'F','PRO':'P','SER':'S','THR':'T','TRP':'W',
       'TYR':'Y','VAL':'V'}

s = parser.get_structure("X", cleaned_pdb_filename)
m = s[0] # model 0
seq = ""
with open(fastaFile, "w") as out:
    for chain in chains:
        out.write(f">{pdb.upper()}:{chain.upper()}|PDBID|CHAIN|SEQUENCE\n")
        c = m[chain]
        chain_seq = ""
        for residue in c:
            residue_name = residue.get_resname()
            chain_seq += ThreeToOne[residue_name]
        out.write("\n".join(textwrap.wrap(chain_seq, width=80))+"\n")
        seq += chain_seq

In [32]:
len(seq)

172

In [33]:
seq

'TSQVRQNYHQDSEAAINRQINLELYASYVYLSMSYYFDRDDVALKNFAKYFLHQSHEEREHAEKLMKLQNQRGGRIFLQDIQKPDCDDWESGLNAMECALHLEKNVNQSLLELHKLATDKNDPHLCDFIETHYLNEQVKAIKELGDHVTNLRKMGAPESGLAEYLFDKHTLG'

In [10]:
pd.read_table(cleaned_pdb_filename, skiprows=skipRows, sep="\s+", names=["ATOM", "i", "Type", "Res", "Chain", "ResId", "x", "y", "z", "_", "_1", "_2"])

,ATOM,i,Type,Res,Chain,ResId,x,y,z,_,_1,_2
0,ATOM,1.0,N,THR,A,1.0,-0.521,-2.002,-0.796,1.0,0.00,N
1,ATOM,2.0,H2,THR,A,1.0,-1.473,-2.223,-0.099,1.0,0.00,H
2,ATOM,3.0,H3,THR,A,1.0,-0.916,-2.230,-1.909,1.0,0.00,H
3,ATOM,4.0,CA,THR,A,1.0,0.097,-0.680,-0.796,1.0,0.00,C
4,ATOM,5.0,C,THR,A,1.0,1.603,-0.786,-0.796,1.0,0.00,C
5,ATOM,6.0,O,THR,A,1.0,2.193,-1.558,-0.068,1.0,0.00,O
6,ATOM,7.0,CB,THR,A,1.0,-0.399,0.159,0.429,1.0,0.00,C
7,ATOM,8.0,CG2,THR,A,1.0,0.182,1.583,0.564,1.0,0.00,C
8,ATOM,9.0,OG1,THR,A,1.0,-1.806,0.353,0.365,1.0,0.00,O
9,ATOM,10.0,H,THR,A,1.0,0.140,-2.971,-0.533,1.0,0.00,H


In [9]:
a

,ATOM,i,Type,Res,Chain,ResId,x,y,z,_,_1,_2
0,ATOM,1.0,N,THR,A,1.0,-0.521,-2.002,-0.796,1.0,0.00,N
1,ATOM,2.0,H2,THR,A,1.0,-1.473,-2.223,-0.099,1.0,0.00,H
2,ATOM,3.0,H3,THR,A,1.0,-0.916,-2.230,-1.909,1.0,0.00,H
3,ATOM,4.0,CA,THR,A,1.0,0.097,-0.680,-0.796,1.0,0.00,C
4,ATOM,5.0,C,THR,A,1.0,1.603,-0.786,-0.796,1.0,0.00,C
5,ATOM,6.0,O,THR,A,1.0,2.193,-1.558,-0.068,1.0,0.00,O
6,ATOM,7.0,CB,THR,A,1.0,-0.399,0.159,0.429,1.0,0.00,C
7,ATOM,8.0,CG2,THR,A,1.0,0.182,1.583,0.564,1.0,0.00,C
8,ATOM,9.0,OG1,THR,A,1.0,-1.806,0.353,0.365,1.0,0.00,O
9,ATOM,10.0,H,THR,A,1.0,0.140,-2.971,-0.533,1.0,0.00,H
